In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant W data

### Reading the files and extracting relevant information:

In [5]:
plant = "W"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  W ['INSTRUÇÕES', 'Clínquer', '140', '196', '163']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('W', '140', array(['CPIIF40'], dtype=object)),
 ('W', '196', array(['CPIIF32'], dtype=object)),
 ('W', '163', array(['CPVARI'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIF40': 1, 'CPIIF32': 1, 'CPVARI': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Composição"].rename(
            {"Sulfato de cálcio": "Sulfato de cálcio (composição)"}, axis=1
        ),
        df["Cimento"]["Análise mineralógica"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [13]:
# df.info()

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [14]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
    "Clínquer": "Clinker",
    "Fíler calcário": "Calcareous filler",
    "Cinza volante": "Fly ash",
    "Escória": "Slag",
    "Argila calcinada": "Calcined clay",
    "Sulfato de cálcio": "Calcium sulfate",
    "C3S total": "Total C3S",
    "C2S total": "Total C2S",
    "C2S alpha": "Alpha C2S",
    "C2S beta": "Beta C2S",
    "C2S gama": "Gamma C2S",
    "C4AF": "C4AF",
    "C3A": "C3A",
    "C3A cubic": "Cubic C3A",
    "C3A orto": "Orthorhombic C3A",
    "CaO livre": "Free CaO",
    "Portlandita": "Portlandite",
    "Periclasio": "Periclase",
    "Arcanita": "Arcanite",
    "Aphthalita": "Aphthitalite",
    "Langbeinita": "Langbeinite",
    "Bassanita": "Bassanite",
    "Anidrita": "Anhydrite",
    "Calcita": "Calcite",
    "Dolomita": "Dolomite",
    "Quartzo": "Quartz",
    "Muscovita": "Muscovite",
    "Sulfato de cálcio (composição)": "Calcium sulfate (composition)",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [15]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "Calcium sulfate (composition)",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Calcium sulfate",
    "Total C3S",
    "Total C2S",
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Langbeinite",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolimita",
    "Quartz",
    "Muscovite",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [18]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
CS1,100.000000
TiO2,100.000000
Specific Gravity,100.000000
Clinker,100.000000
Calcareous filler,100.000000
Fly ash,100.000000
Slag,100.000000
Calcined clay,100.000000
Langbeinite,100.000000
Calcium sulfate,100.000000


<IPython.core.display.Javascript object>

In [19]:
for var, val in (
    (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).items()
):
    s = f"""\t<tr>
        <td>{var}</td>
        <td>{round(val, 2)}</td>
    \t</tr>"""
    if val >= 40:
        # print(s)
        print(f'"{var}",')

"CS1",
"TiO2",
"Specific Gravity",
"Clinker",
"Calcareous filler",
"Fly ash",
"Slag",
"Calcined clay",
"Langbeinite",
"Calcium sulfate",
"Calcium sulfate (composition)",
"Total C2S",
"#400",


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant W - Full Dataset:</th>
    <th></th>
	<tr>
        <td>CS1</td>
        <td>100.0</td>
    	</tr>
	<tr>
        <td>TiO2</td>
        <td>100.0</td>
    	</tr>
	<tr>
        <td>Specific Gravity</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Clinker</td>
        <td>100.0</td>
    	</tr>
	<tr>
        <td>Calcareous filler</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Fly ash</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Slag</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Calcined clay</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Langbeinite</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Calcium sulfate</td>
        <td>100.0</td>
    </tr>
	<tr>
        <td>Calcium sulfate (composition)</td>
        <td>49.91</td>
    	</tr>
	<tr>
        <td>Total C2S</td>
        <td>43.23</td>
    	</tr>
	<tr>
        <td>#400</td>
        <td>42.96</td>
    </tr>
</table>

In [20]:
FEATRUES_TO_DROP = [
    "CS1",
    "TiO2",
    "Specific Gravity",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Langbeinite",
    "Calcium sulfate",
    "Calcium sulfate (composition)",
    "Total C2S",
    "#400",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with more than 23% of zeros

In [21]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for W
Muscovite,91.889483
Gamma C2S,53.654189
Anhydrite,28.163993
#200,18.805704
Arcanite,4.456328
Alpha C2S,1.426025
Bassanite,0.713012
Dolimita,0.356506
Free CaO,0.267380
Aphthitalite,0.267380


<IPython.core.display.Javascript object>

<h3>Percentage of zero values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Muscovite</td>
        <td>91.89</td>
    </tr>
    <tr>
        <td>Gamma C2S</td>
        <td>53.65</td>
    </tr>
    <tr>
        <td>Anhydrite</td>
        <td>28.16</td>
    </tr>    
    <tr>
        <td>#200</td>
        <td>18.81</td>
    </tr>
</table>

In [22]:
FEATRUES_TO_DROP = ["Muscovite", "Gamma C2S", "Anhydrite", "#200"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [23]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [24]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [25]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [26]:
df.shape

(1088, 35)

<IPython.core.display.Javascript object>

In [27]:
# df.describe().T

<IPython.core.display.Javascript object>

In [28]:
# df_copy = df.copy()

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [29]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [30]:
df.shape

(1011, 35)

<IPython.core.display.Javascript object>

In [31]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Dolimita,7.319486
Portlandite,7.319486
Total C3S,7.319486
Quartz,7.319486
Calcite,7.319486
Bassanite,7.319486
Aphthitalite,7.319486
Arcanite,7.319486
Periclase,7.319486
Free CaO,7.319486


<IPython.core.display.Javascript object>

## Dropping rows with increased missing values after preprocessing

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant W - Full Dataset:</th>
    <th></th>
    <tr>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td></td>
        <td></td>
    </tr>
</table>

In [32]:
# FEATRUES_TO_DROP = ["C3A", "Aphthitalite"]
# df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [33]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Dolimita,7.319486
Portlandite,7.319486
Total C3S,7.319486
Quartz,7.319486
Calcite,7.319486
Bassanite,7.319486
Aphthitalite,7.319486
Arcanite,7.319486
Periclase,7.319486
Free CaO,7.319486


<IPython.core.display.Javascript object>

In [34]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1011,2022-04-27 03:55:00.890207744,2021-01-04 00:00:00,2021-09-17 12:00:00,2022-05-04 00:00:00,2022-12-11 00:00:00,2023-07-25 00:00:00,NaN
Total C3S,937.0,51.832775,38.02,46.41,53.19,56.16,64.8,5.706461
Alpha C2S,937.0,1.930416,0.03,1.25,1.93,2.58,5.38,0.934011
Beta C2S,937.0,8.381473,3.38,7.31,8.32,9.25,13.35,1.496824
C4AF,937.0,8.27444,5.22,7.6,8.3,8.94,11.31,1.039782
C3A,937.0,3.891089,2.2,3.37,3.77,4.39,6.62,0.703047
Cubic C3A,937.0,3.146681,1.48,2.78,3.2,3.53,4.68,0.530595
Orthorhombic C3A,937.0,0.741345,0.01,0.45,0.67,1.0,3.12,0.408527
Free CaO,937.0,0.630971,0.03,0.37,0.59,0.82,2.23,0.334067
Portlandite,937.0,0.876766,0.09,0.56,0.75,1.02,4.79,0.513585


<IPython.core.display.Javascript object>

In [35]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,1011,NaN,NaN,NaN,2022-04-27 03:55:00.890207744,2021-01-04 00:00:00,2021-09-17 12:00:00,2022-05-04 00:00:00,2022-12-11 00:00:00,2023-07-25 00:00:00
Cement_Type,1011,3,CP II-F-40,600,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1011 entries, 0 to 1119
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1011 non-null   datetime64[ns]
 1   Total C3S             937 non-null    float64       
 2   Alpha C2S             937 non-null    float64       
 3   Beta C2S              937 non-null    float64       
 4   C4AF                  937 non-null    float64       
 5   C3A                   937 non-null    float64       
 6   Cubic C3A             937 non-null    float64       
 7   Orthorhombic C3A      937 non-null    float64       
 8   Free CaO              937 non-null    float64       
 9   Portlandite           937 non-null    float64       
 10  Periclase             937 non-null    float64       
 11  Arcanite              937 non-null    float64       
 12  Aphthitalite          937 non-null    float64       
 13  Bassanite             9

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [37]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [38]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [39]:
df.to_csv("../../../data/interim/209/w.csv", index=False)

<IPython.core.display.Javascript object>